In [10]:
import pandas as pd
import gradio as gr


def get_BFCs(results_df):
    df = results_df[['hash', 'bfc']]
    return df

results_A = pd.read_csv('annotations_Michel.csv')
results_B = pd.read_csv('annotations_Abhishek.csv')
results_C = pd.read_csv('annotations_David.csv')

BFCs_A = get_BFCs(results_A)
BFCs_B = get_BFCs(results_B)
BFCs_C = get_BFCs(results_C)

BFCs = pd.merge(BFCs_A, BFCs_B, on='hash', how='inner', suffixes=('A', 'B'))

BFCs = pd.merge(BFCs, BFCs_C, on='hash', how='inner', suffixes=('', 'C'))
BFCs = BFCs[['hash', 'bfcA', 'bfcB', 'bfc']]
BFCs = BFCs.rename(columns={'bfc': 'bfcC'})

In [8]:
BFCs_R = pd.read_csv('review.csv')
BFCs = pd.merge(BFCs, BFCs_R, on='hash', how='inner')
BFCs['bfcR'] = BFCs['bfc']
BFCs = BFCs[['hash', 'bfcA', 'bfcB', 'bfcC', 'bfcR']]

In [11]:
BFCs

,hash,bfcA,bfcB,bfcC
0,0704a8586f,0,0,0
1,c5e97ed154,0,0,0
2,4f9f531e15,0,0,0
3,258030acc9,0,0,0
4,5b2c5540b8,4,4,3
...,...,...,...,...
504,38d4a385a3,0,0,0
505,90b13cdde1,0,0,0
506,ebca1b8056,0,0,0
507,ec1d7e6ab9,0,0,0


In [12]:
def compute_disagreement(BFCs):
    BFCs_disagree = BFCs.query("(bfcA != bfcB) or (bfcA != bfcC)")
    total_disagree = len(BFCs_disagree)
    print(f"Total commits with disagreement: {total_disagree}")
    return BFCs_disagree

BFCs_disagree = compute_disagreement(BFCs)
BFCs_disagree

Total commits with disagreement: 102


,hash,bfcA,bfcB,bfcC
4,5b2c5540b8,4,4,3
15,9a10064f56,3,0,3
18,016017a195,3,4,3
19,11c2bf4a20,2,4,2
22,088c840599,0,4,0
...,...,...,...,...
465,cc1049ccee,3,4,4
470,9f22f95951,3,4,0
475,aa303964bc,1,0,0
478,cff35798fa,4,0,0


In [13]:
BFCs_disagree_1 = BFCs.query("(abs(bfcA - bfcB) > 1) or (abs(bfcA - bfcC) > 1)")
total_disagree_1 = len(BFCs_disagree_1)
print(f"Total commits with disagreement: {total_disagree_1}")
#BFCs_disagree_1

Total commits with disagreement: 65


In [14]:
def reduce3(value):
    if type(value) is not int:
        return value
    if value <= 1:
        return 0
    elif value >= 3:
        return 2
    else:
        return 1

BFCs3 = BFCs.map(reduce3)
#BFCs3

In [15]:
BFCs3_disagree = compute_disagreement(BFCs3)
#BFCs3_disagree

Total commits with disagreement: 69


In [16]:
def disagreement_matrix(BFCs_disagree):
    persons = ['A', 'B', 'C', 'R']
    print("    ", "    ".join(persons))
    for row in persons:
        print(f"{row}", end=" ")
        for column in persons:
            if column > row:
                dis = len(BFCs_disagree.query(f"(bfc{column} != bfc{row})"))
                print(f"{dis:>4}", end=" ")
            else:
                print("    ", end=" ")
        print()
disagreement_matrix(BFCs_disagree)

     A    B    C    R
A        91   65 

UndefinedVariableError: name 'bfcR' is not defined

In [21]:
disagreement_matrix(BFCs3_disagree)

     A    B    C    R
A        19   19   15 
B             22   17 
C                   9 
R                     


# Agreement as a distance between values
def compute_dist(df, persons):
    agree_dist = pd.DataFrame()
    for index1 in range(0, len(persons)-1):
        for index2 in range(index1+1,len(persons)):
            label1 = persons[index1]
            label2 = persons[index2]
            agree_dist[f'{label1}-{label2}'] = abs(BFCs[f'bfc{label1}'] - BFCs[f'bfc{label2}'])    
    return agree_dist
    
agree_dist = compute_dist(df=BFCs, persons=['A', 'B', 'C', 'R'])

In [38]:
#agree_dist

,hash,bfcA,bfcB,bfcC,bfcR
0,974d5ef0dd,0,4,0,0
1,e2d8ea4320,0,4,0,0
2,d7d7ddc156,1,3,0,0
3,b1c87b0874,4,4,1,1
4,088604d37e,4,4,1,0
5,e6fc9f62ce,2,4,0,1
6,d2ed913b9a,3,4,0,0
7,3648bcf1c1,3,0,0,0
8,fbb1d4b381,4,2,0,0
9,4528668ca3,0,4,3,0


In [45]:
def compute_dist_matrix(df, persons):
    print("    ", "    ".join(persons))
    for row in persons:
        print(f"{row}", end=" ")
        for column in persons:
            if column > row:
                dist = abs(BFCs[f'bfc{column}'] - BFCs[f'bfc{row}']).mean()
                print(f"{dist:.2f}", end=" ")
            else:
                print("    ", end=" ")
        print()
compute_dist_matrix(df=BFCs, persons=['A', 'B', 'C', 'R'])


     A    B    C    R
A      1.96 1.79 1.61 
B           2.46 2.07 
C                1.04 
R                     
